<h1><center>MPST: A Corpus of Movie Plot Synopses with Tags</center></h1>

In [1]:
!pip install scikit-multilearn

import re
import os
import tqdm
import nltk
import pickle
import sqlite3
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import xgboost as xgb
import tensorflow as tf
from sklearn import metrics
from tensorflow import keras
from nltk.corpus import words
from datetime import datetime
from bs4 import BeautifulSoup
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from itertools import combinations
from keras.models import load_model
from keras.models import Sequential
from tensorflow.keras import layers
from nltk.stem import SnowballStemmer
from sklearn.pipeline import Pipeline
from nltk.tokenize import sent_tokenize
from keras.preprocessing import sequence
from scipy.sparse import coo_matrix, hstack
from tensorflow.keras.utils import plot_model
from keras.layers.embeddings import Embedding
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import RandomizedSearchCV
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import f1_score,precision_score,recall_score,hamming_loss
from keras.layers import Conv1D, Conv2D, Dense, Dropout, Flatten, LSTM, GlobalMaxPooling1D, MaxPooling2D, Activation, BatchNormalization

%matplotlib inline
nltk.download('punkt')
nltk.download('wordnet')
warnings.filterwarnings("ignore")
stemmer = SnowballStemmer('english')

%autosave 120

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bhanu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bhanu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Autosaving every 120 seconds


In [2]:
data_with_all_tags = pd.read_csv("data_with_all_tags.csv")
data_with_all_tags.head()

,title,plot_synopsis,tags,split,synopsis_source,cnt_dup,tag_count,synopsis_count,synopsis_sent_count,CleanedSynopsis,...,sentimental,storytelling,stupid,suicidal,suspenseful,thought-provoking,tragedy,violence,western,whimsical
0,$,"Set in Hamburg, West Germany, several criminal...",murder,test,imdb,1,1,648,26,set hamburg west germani sever crimin take adv...,...,0,0,0,0,0,0,0,0,0,0
1,$windle,A 6th grader named Griffin Bing decides to gat...,flashback,train,wikipedia,1,1,353,14,grader name griffin bing decid gather entir gr...,...,0,0,0,0,0,0,0,0,0,0
2,'71,"Gary Hook, a new recruit to the British Army, ...","suspenseful, neo noir, murder, violence",train,wikipedia,1,4,699,39,gari hook new recruit british armi take leav m...,...,0,0,0,0,1,0,0,1,0,0
3,'A' gai wak,Sergeant Dragon Ma (Jackie Chan) is part of th...,"cult, violence",train,wikipedia,1,2,665,41,sergeant dragon jacki chan part hong kong mari...,...,0,0,0,0,0,0,0,1,0,0
4,'Breaker' Morant,"In Pretoria, South Africa, in 1902, Major Char...","murder, anti war, violence, flashback, tragedy...",train,wikipedia,1,6,1694,140,pretoria south africa major charl bolton rod m...,...,0,0,0,0,0,0,1,1,0,0


In [3]:
conn = sqlite3.connect('data.db')
data_with_all_tags.to_sql('data', conn, if_exists='replace', index=False)
train = pd.read_sql("Select * From data where split = 'train' OR split='val'",conn)
test =  pd.read_sql("Select * From data where split = 'test'",conn)
conn.close()

In [4]:
X_train = train["CleanedSynopsis"]
y_train= train["tags"]

X_test = test["CleanedSynopsis"]
y_test= test["tags"]

In [5]:
def tokenize(x):
    x=x.split(',')
    tags=[i.strip() for i in x] #Some tags contains whitespaces before them, so we need to strip them
    return tags

cnt_vectorizer = CountVectorizer(tokenizer = tokenize, max_features=5, binary='true').fit(y_train)
y_train_multilabel = cnt_vectorizer.transform(y_train)
y_test_multilabel = cnt_vectorizer.transform(y_test)

<h1> 1. TfidfVectorizer with 1 grams: </h1>

In [6]:
tf_vectorizer = TfidfVectorizer(min_df=0.09, tokenizer = lambda x: x.split(" "), ngram_range=(1,1))

X_train_multilabel = tf_vectorizer.fit_transform(X_train)
X_test_multilabel = tf_vectorizer.transform(X_test)

print("Dimensions of train data X:",X_train_multilabel.shape, "Y :",y_train_multilabel.shape)
print("Dimensions of test data X:",X_test_multilabel.shape,"Y:",y_test_multilabel.shape)

Dimensions of train data X: (10989, 657) Y : (10989, 5)
Dimensions of test data X: (2768, 657) Y: (2768, 5)


<h2> 1.1 OneVsRestClassifier + MultinomialNB:</h2>

In [7]:
mb = MultinomialNB(class_prior = [0.5, 0.5])

clf = OneVsRestClassifier(mb)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=MultinomialNB(class_prior=[0.5, 0.5]))

In [8]:
prediction1 = clf.predict(X_test_multilabel)

precision1 = precision_score(y_test_multilabel, prediction1, average='micro')

recall1 = recall_score(y_test_multilabel, prediction1, average='micro')

f1_score1 = 2*((precision1 * recall1)/(precision1 + recall1))

print("precision1: {:.4f}, recall1: {:.4f}, F1-measure: {:.4f}".format(precision1, recall1, f1_score1))

precision1: 0.4110, recall1: 0.6757, F1-measure: 0.5111


In [9]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction1[k])[0],"\n")

Movie:  White Tiger
Actual genre:  murder
Predicted tag:  ['violence'] 

Movie:  The Upturned Glass
Actual genre:  revenge, murder
Predicted tag:  ['flashback' 'murder'] 

Movie:  Séraphine
Actual genre:  depressing
Predicted tag:  ['romantic'] 

Movie:  Two Weeks Notice
Actual genre:  romantic, entertaining
Predicted tag:  ['romantic'] 

Movie:  Ghilli
Actual genre:  murder
Predicted tag:  ['romantic'] 



<h2> 1.2 OneVsRestClassifier + SGDClassifier with LOG Loss:</h2>

In [10]:
sgl = SGDClassifier(loss='log', class_weight='balanced')

clf = OneVsRestClassifier(sgl)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(class_weight='balanced',
                                            loss='log'))

In [11]:
prediction2 = clf.predict(X_test_multilabel)

precision2 = precision_score(y_test_multilabel, prediction2, average='micro')

recall2 = recall_score(y_test_multilabel, prediction2, average='micro')

f1_score2 = 2*((precision2 * recall2)/(precision2 + recall2))

print("precision2: {:.4f}, recall2: {:.4f}, F1-measure: {:.4f}".format(precision2, recall2, f1_score2))

precision2: 0.4478, recall2: 0.6760, F1-measure: 0.5387


In [12]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction2[k])[0],"\n")

Movie:  La vieille dame et les pigeons
Actual genre:  insanity, psychedelic, comedy
Predicted tag:  ['cult' 'violence'] 

Movie:  Gunsight Ridge
Actual genre:  violence, murder
Predicted tag:  ['murder' 'violence'] 

Movie:  Exodus: Gods and Kings
Actual genre:  comedy, cruelty, murder, violence, psychedelic, revenge
Predicted tag:  ['flashback' 'murder' 'violence'] 

Movie:  Tammy and the Bachelor
Actual genre:  romantic, storytelling
Predicted tag:  ['romantic'] 

Movie:  Earth Angel
Actual genre:  romantic
Predicted tag:  ['flashback' 'romantic'] 



<h2> 1.3 OneVsRestClassifier + SGDClassifier with Hinge Loss:</h2>

In [13]:
sgh = SGDClassifier(loss='hinge', class_weight='balanced')

clf = OneVsRestClassifier(sgh)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(class_weight='balanced'))

In [14]:
prediction3 = clf.predict(X_test_multilabel)

precision3 = precision_score(y_test_multilabel, prediction3, average='micro')

recall3 = recall_score(y_test_multilabel, prediction3, average='micro')

f1_score3 = 2*((precision3 * recall3)/(precision3 + recall3))

print("precision3: {:.4f}, recall3: {:.4f}, F1-measure: {:.4f}".format(precision3, recall3, f1_score3))

precision3: 0.4352, recall3: 0.6737, F1-measure: 0.5288


In [15]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction3[k])[0],"\n")

Movie:  Kill List
Actual genre:  mystery, murder, cult, horror, violence, flashback
Predicted tag:  ['cult' 'flashback' 'murder' 'violence'] 

Movie:  Outrage
Actual genre:  romantic, psychological, violence, queer
Predicted tag:  ['murder'] 

Movie:  Kenka erejî
Actual genre:  satire, comic
Predicted tag:  ['romantic'] 

Movie:  Aerial Gunner
Actual genre:  romantic, flashback
Predicted tag:  ['murder' 'romantic' 'violence'] 

Movie:  The Possession of Joel Delaney
Actual genre:  violence, murder, flashback
Predicted tag:  ['cult' 'flashback' 'murder' 'violence'] 



<h2> 1.4  OneVsRestClassifier + LogisticRegression:</h2>

In [16]:
lr = LogisticRegression(class_weight='balanced')

clf = OneVsRestClassifier(lr)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=LogisticRegression(class_weight='balanced'))

In [17]:
prediction4 = clf.predict(X_test_multilabel)

precision4 = precision_score(y_test_multilabel, prediction4, average='micro')

recall4 = recall_score(y_test_multilabel, prediction4, average='micro')

f1_score4 = 2*((precision4 * recall4)/(precision4 + recall4))

print("precision4: {:.4f}, recall4: {:.4f}, F1-measure: {:.4f}".format(precision4, recall4, f1_score4))

precision4: 0.4469, recall4: 0.6679, F1-measure: 0.5355


In [18]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction4[k])[0],"\n")

Movie:  The Creeping Flesh
Actual genre:  murder
Predicted tag:  ['flashback' 'murder'] 

Movie:  Wong Fei Hung III: Si wong jaang ba
Actual genre:  cult, violence
Predicted tag:  ['cult' 'romantic'] 

Movie:  La cérémonie
Actual genre:  pornographic, sadist
Predicted tag:  ['flashback' 'romantic'] 

Movie:  Valmont
Actual genre:  tragedy, revenge, murder, romantic
Predicted tag:  ['romantic'] 

Movie:  13 Tzameti
Actual genre:  dramatic, revenge, mystery
Predicted tag:  ['flashback' 'murder' 'violence'] 



<h1>2. TfidfVectorizer with (1 - 2 Grams):<?h1>

In [19]:
tf_vectorizer = TfidfVectorizer(min_df=0.09, tokenizer = lambda x: x.split(" "), ngram_range=(1,2))

X_train_multilabel = tf_vectorizer.fit_transform(X_train)
X_test_multilabel = tf_vectorizer.transform(X_test)

print("Dimensions of train data X:",X_train_multilabel.shape, "Y :",y_train_multilabel.shape)
print("Dimensions of test data X:",X_test_multilabel.shape,"Y:",y_test_multilabel.shape)

Dimensions of train data X: (10989, 666) Y : (10989, 5)
Dimensions of test data X: (2768, 666) Y: (2768, 5)


<H2> 2.1 OneVsRestClassifier + MultinomialNB :</H2>

In [20]:
mb = MultinomialNB(class_prior = [0.5, 0.5])

clf = OneVsRestClassifier(mb)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=MultinomialNB(class_prior=[0.5, 0.5]))

In [21]:
prediction5 = clf.predict(X_test_multilabel)

precision5 = precision_score(y_test_multilabel, prediction5, average='micro')

recall5 = recall_score(y_test_multilabel, prediction5, average='micro')

f1_score5 = 2*((precision5 * recall5)/(precision5 + recall5))

print("precision5: {:.4f}, recall5: {:.4f}, F1-measure: {:.4f}".format(precision5, recall5, f1_score5))

precision5: 0.4120, recall5: 0.6762, F1-measure: 0.5120


In [22]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction5[k])[0],"\n")

Movie:  Der bewegte Mann
Actual genre:  comedy
Predicted tag:  ['romantic'] 

Movie:  Keith Lemon: The Film
Actual genre:  comedy
Predicted tag:  ['romantic'] 

Movie:  Emma
Actual genre:  romantic, satire
Predicted tag:  ['romantic'] 

Movie:  Enemy of the State
Actual genre:  comedy, mystery, neo noir, murder, cult, revenge, violence, humor, suspenseful
Predicted tag:  ['cult' 'flashback' 'murder' 'violence'] 

Movie:  Desperate Remedies
Actual genre:  comedy, gothic, murder, cult, intrigue, violence, melodrama
Predicted tag:  ['romantic'] 



<h2> 2.2 OneVsRestClassifier + SGDClassifier with LOG Loss :</h2>

In [23]:
sgl = SGDClassifier(loss='log', class_weight='balanced')

clf = OneVsRestClassifier(sgl)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(class_weight='balanced',
                                            loss='log'))

In [24]:
prediction6 = clf.predict(X_test_multilabel)

precision6 = precision_score(y_test_multilabel, prediction6, average='micro')

recall6 = recall_score(y_test_multilabel, prediction6, average='micro')

f1_score6 = 2*((precision6 * recall6)/(precision6 + recall6))

print("precision6: {:.4f}, recall6: {:.4f}, F1-measure: {:.4f}".format(precision6, recall6, f1_score6))

precision6: 0.4549, recall6: 0.6603, F1-measure: 0.5387


In [25]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction6[k])[0],"\n")

Movie:  Easy Rider
Actual genre:  comedy, avant garde, dark, murder, realism, cult, atmospheric, psychedelic, humor, satire
Predicted tag:  ['cult'] 

Movie:  Better Living Through Chemistry
Actual genre:  entertaining, clever
Predicted tag:  ['romantic'] 

Movie:  47 Ronin
Actual genre:  fantasy, murder, paranormal, violence, historical fiction, flashback, psychedelic, romantic, tragedy, revenge
Predicted tag:  ['cult' 'flashback' 'violence'] 

Movie:  0.45
Actual genre:  murder, flashback
Predicted tag:  ['murder' 'romantic'] 

Movie:  The Sugarland Express
Actual genre:  violence, neo noir
Predicted tag:  ['cult' 'flashback' 'murder'] 



<h2> 2.3 OneVsRestClassifier + SGDClassifier with HINGE Loss : </h2>

In [26]:
sgh = SGDClassifier(loss='hinge', class_weight='balanced')

clf = OneVsRestClassifier(sgh)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(class_weight='balanced'))

In [27]:
prediction7 = clf.predict(X_test_multilabel)

precision7 = precision_score(y_test_multilabel, prediction7, average='micro')

recall7 = recall_score(y_test_multilabel, prediction7, average='micro')

f1_score7 = 2*((precision7 * recall7)/(precision7 + recall7))

print("precision7: {:.4f}, recall7: {:.4f}, F1-measure: {:.4f}".format(precision7, recall7, f1_score7))

precision7: 0.4415, recall7: 0.6723, F1-measure: 0.5330


In [28]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction7[k])[0],"\n")

Movie:  Next
Actual genre:  paranormal, plot twist, action, murder
Predicted tag:  ['cult' 'violence'] 

Movie:  Mimi wo sumaseba
Actual genre:  inspiring, psychedelic, romantic
Predicted tag:  ['flashback' 'romantic'] 

Movie:  Devil's Due
Actual genre:  paranormal, dark, cult, murder
Predicted tag:  ['cult' 'murder'] 

Movie:  Airman
Actual genre:  murder
Predicted tag:  ['romantic'] 

Movie:  Labor Pains
Actual genre:  romantic
Predicted tag:  ['romantic'] 



<h2> 2.4 OneVsRestClassifier + LogisticRegression:</h2>

In [29]:
lr = LogisticRegression(class_weight='balanced')

clf = OneVsRestClassifier(lr)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=LogisticRegression(class_weight='balanced'))

In [30]:
prediction8 = clf.predict(X_test_multilabel)

precision8 = precision_score(y_test_multilabel, prediction8, average='micro')

recall8 = recall_score(y_test_multilabel, prediction8, average='micro')

f1_score8 = 2*((precision8 * recall8)/(precision8 + recall8))

print("precision8: {:.4f}, recall8: {:.4f}, F1-measure: {:.4f}".format(precision8, recall8, f1_score8))

precision8: 0.4469, recall8: 0.6676, F1-measure: 0.5354


In [31]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction8[k])[0],"\n")

Movie:  The East
Actual genre:  suspenseful, neo noir, violence, tragedy, revenge, sentimental
Predicted tag:  ['murder'] 

Movie:  When the Wind Blows
Actual genre:  tragedy, comedy, storytelling, flashback
Predicted tag:  [] 

Movie:  Codebreaker
Actual genre:  tragedy
Predicted tag:  ['flashback'] 

Movie:  Constantine
Actual genre:  comedy, fantasy, gothic, murder, paranormal, cult, flashback, good versus evil, psychedelic, storytelling
Predicted tag:  ['cult' 'flashback' 'violence'] 

Movie:  Tale of a Vampire
Actual genre:  violence
Predicted tag:  ['romantic'] 



<h1>3. TfidfVectorizer with (1 - 3 Grams):<?h1>

In [32]:
tf_vectorizer = TfidfVectorizer(min_df=0.09, tokenizer = lambda x: x.split(" "), ngram_range=(1,3))

X_train_multilabel = tf_vectorizer.fit_transform(X_train)
X_test_multilabel = tf_vectorizer.transform(X_test)

print("Dimensions of train data X:",X_train_multilabel.shape, "Y :",y_train_multilabel.shape)
print("Dimensions of test data X:",X_test_multilabel.shape,"Y:",y_test_multilabel.shape)

Dimensions of train data X: (10989, 666) Y : (10989, 5)
Dimensions of test data X: (2768, 666) Y: (2768, 5)


<H2> 3.1 OneVsRestClassifier + MultinomialNB :</H2>

In [33]:
mb = MultinomialNB(class_prior = [0.5, 0.5])

clf = OneVsRestClassifier(mb)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=MultinomialNB(class_prior=[0.5, 0.5]))

In [34]:
prediction9 = clf.predict(X_test_multilabel)

precision9 = precision_score(y_test_multilabel, prediction9, average='micro')

recall9 = recall_score(y_test_multilabel, prediction9, average='micro')

f1_score9 = 2*((precision9 * recall9)/(precision9 + recall9))

print("precision9: {:.4f}, recall9: {:.4f}, F1-measure: {:.4f}".format(precision9, recall9, f1_score9))

precision9: 0.4120, recall9: 0.6762, F1-measure: 0.5120


In [35]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction9[k])[0],"\n")

Movie:  Tom Tom Tomcat
Actual genre:  psychedelic
Predicted tag:  ['cult' 'violence'] 

Movie:  The Giver
Actual genre:  violence, cruelty, murder, flashback
Predicted tag:  ['flashback' 'romantic'] 

Movie:  The Death Kiss
Actual genre:  murder
Predicted tag:  ['cult' 'flashback' 'murder' 'violence'] 

Movie:  Shock
Actual genre:  romantic
Predicted tag:  ['flashback' 'murder'] 

Movie:  Tinker Bell and the Lost Treasure
Actual genre:  fantasy
Predicted tag:  ['cult'] 



<H2> 3.2 OneVsRestClassifier + SGDClassifier with LOG Loss :</H2>

In [36]:
sgl = SGDClassifier(loss='log', class_weight='balanced')

clf = OneVsRestClassifier(sgl)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(class_weight='balanced',
                                            loss='log'))

In [37]:
prediction10 = clf.predict(X_test_multilabel)

precision10 = precision_score(y_test_multilabel, prediction10, average='micro')

recall10 = recall_score(y_test_multilabel, prediction10, average='micro')

f1_score10 = 2*((precision10 * recall10)/(precision10 + recall10))

print("precision10: {:.4f}, recall10: {:.4f}, F1-measure: {:.4f}".format(precision10, recall10, f1_score10))

precision10: 0.4438, recall10: 0.6648, F1-measure: 0.5323


In [38]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction10[k])[0],"\n")

Movie:  Las brujas de Zugarramurdi
Actual genre:  romantic
Predicted tag:  ['cult' 'violence'] 

Movie:  Gunfight at the O.K. Corral
Actual genre:  cult, historical
Predicted tag:  ['flashback' 'murder' 'violence'] 

Movie:  The Upturned Glass
Actual genre:  revenge, murder
Predicted tag:  ['flashback' 'murder' 'romantic'] 

Movie:  Baby, Take a Bow
Actual genre:  melodrama
Predicted tag:  ['flashback' 'murder'] 

Movie:  The Young Savages
Actual genre:  murder
Predicted tag:  ['murder' 'violence'] 



<h2> 3.3 OneVsRestClassifier + SGDClassifier with HINGE Loss : </h2>

In [39]:
sgh = SGDClassifier(loss='hinge', class_weight='balanced')

clf = OneVsRestClassifier(sgh)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(class_weight='balanced'))

In [40]:
prediction11 = clf.predict(X_test_multilabel)

precision11 = precision_score(y_test_multilabel, prediction11, average='micro')

recall11 = recall_score(y_test_multilabel, prediction11, average='micro')

f1_score11 = 2*((precision11 * recall11)/(precision11 + recall11))

print("precision11: {:.4f}, recall11: {:.4f}, F1-measure: {:.4f}".format(precision11, recall11, f1_score11))

precision11: 0.4336, recall11: 0.6679, F1-measure: 0.5258


In [41]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction11[k])[0],"\n")

Movie:  The Rat Race
Actual genre:  romantic
Predicted tag:  ['romantic'] 

Movie:  Gingerdead Man 2: Passion of the Crust
Actual genre:  comedy, murder, violence
Predicted tag:  ['cult' 'flashback' 'murder' 'violence'] 

Movie:  Picnic at Hanging Rock
Actual genre:  mystery, historical, boring, flashback
Predicted tag:  ['cult' 'flashback' 'murder'] 

Movie:  So weit die Füße tragen
Actual genre:  violence, cruelty, sadist, historical fiction
Predicted tag:  [] 

Movie:  A Time to Kill
Actual genre:  tragedy, revenge, murder, violence
Predicted tag:  ['murder' 'violence'] 



<h2> 3.4 OneVsRestClassifier + LogisticRegression:</h2>

In [42]:
lr = LogisticRegression(class_weight='balanced')

clf = OneVsRestClassifier(lr)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=LogisticRegression(class_weight='balanced'))

In [43]:
prediction12 = clf.predict(X_test_multilabel)

precision12 = precision_score(y_test_multilabel, prediction12, average='micro')

recall12 = recall_score(y_test_multilabel, prediction12, average='micro')

f1_score12 = 2*((precision12 * recall12)/(precision12 + recall12))

print("precision12: {:.4f}, recall12: {:.4f}, F1-measure: {:.4f}".format(precision12, recall12, f1_score12))

precision12: 0.4469, recall12: 0.6676, F1-measure: 0.5354


In [44]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction12[k])[0],"\n")

Movie:  Bad Kids Go to Hell
Actual genre:  revenge, comedy, murder, flashback
Predicted tag:  ['cult' 'flashback' 'murder'] 

Movie:  The Duellists
Actual genre:  violence, action, historical
Predicted tag:  ['flashback' 'romantic' 'violence'] 

Movie:  Absence of Malice
Actual genre:  murder
Predicted tag:  ['murder'] 

Movie:  Sea Devils
Actual genre:  historical
Predicted tag:  ['cult'] 

Movie:  Mr. Holmes
Actual genre:  storytelling, flashback
Predicted tag:  ['flashback' 'romantic'] 



<h1>4. TfidfVectorizer with (1 - 4 Grams):</h1>

In [45]:
tf_vectorizer = TfidfVectorizer(min_df=0.09, tokenizer = lambda x: x.split(" "), ngram_range=(1, 4))

X_train_multilabel = tf_vectorizer.fit_transform(X_train)
X_test_multilabel = tf_vectorizer.transform(X_test)

print("Dimensions of train data X:",X_train_multilabel.shape, "Y :",y_train_multilabel.shape)
print("Dimensions of test data X:",X_test_multilabel.shape,"Y:",y_test_multilabel.shape)

Dimensions of train data X: (10989, 666) Y : (10989, 5)
Dimensions of test data X: (2768, 666) Y: (2768, 5)


<H2> 4.1 OneVsRestClassifier + MultinomialNB :</H2>

In [46]:
mb = MultinomialNB(class_prior = [0.5, 0.5])

clf = OneVsRestClassifier(mb)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=MultinomialNB(class_prior=[0.5, 0.5]))

In [47]:
prediction13 = clf.predict(X_test_multilabel)

precision13 = precision_score(y_test_multilabel, prediction13, average='micro')

recall13 = recall_score(y_test_multilabel, prediction13, average='micro')

f1_score13 = 2*((precision13 * recall13)/(precision13 + recall13))

print("precision13: {:.4f}, recall13: {:.4f}, F1-measure: {:.4f}".format(precision13, recall13, f1_score13))

precision13: 0.4120, recall13: 0.6762, F1-measure: 0.5120


In [48]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction13[k])[0],"\n")

Movie:  The Spook Who Sat by the Door
Actual genre:  violence, cult, avant garde, murder, blaxploitation
Predicted tag:  ['violence'] 

Movie:  Scapegoat
Actual genre:  revenge, suspenseful
Predicted tag:  ['murder' 'romantic'] 

Movie:  Home in Indiana
Actual genre:  romantic
Predicted tag:  ['romantic'] 

Movie:  Wagon Master
Actual genre:  murder
Predicted tag:  ['romantic'] 

Movie:  Dark Passage
Actual genre:  murder, atmospheric
Predicted tag:  ['cult' 'murder' 'violence'] 



<h2> 4.2 OneVsRestClassifier + SGDClassifier with LOG Loss :</h2>

In [49]:
sgl = SGDClassifier(loss='log', class_weight='balanced')

clf = OneVsRestClassifier(sgl)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(class_weight='balanced',
                                            loss='log'))

In [50]:
prediction14 = clf.predict(X_test_multilabel)

precision14 = precision_score(y_test_multilabel, prediction14, average='micro')

recall14 = recall_score(y_test_multilabel, prediction14, average='micro')

f1_score14 = 2*((precision14 * recall14)/(precision14 + recall14))

print("precision14: {:.4f}, recall14: {:.4f}, F1-measure: {:.4f}".format(precision14, recall14, f1_score14))

precision14: 0.4448, recall14: 0.6746, F1-measure: 0.5361


In [51]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction14[k])[0],"\n")

Movie:  Teenage Caveman
Actual genre:  pornographic
Predicted tag:  ['cult' 'murder'] 

Movie:  Paul
Actual genre:  humor, psychedelic, comedy, murder, flashback
Predicted tag:  ['cult' 'violence'] 

Movie:  Yadon ilaheyya
Actual genre:  romantic, satire
Predicted tag:  ['cult' 'violence'] 

Movie:  Letters from Iwo Jima
Actual genre:  boring, murder, dramatic, violence, flashback, claustrophobic, sentimental
Predicted tag:  ['violence'] 

Movie:  The Bucket List
Actual genre:  romantic, comedy, philosophical, boring
Predicted tag:  ['romantic'] 



<h2> 4.3 OneVsRestClassifier + SGDClassifier with HINGE Loss : </h2>

In [52]:
sgh = SGDClassifier(loss='hinge', class_weight='balanced')

clf = OneVsRestClassifier(sgh)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(class_weight='balanced'))

In [53]:
prediction15 = clf.predict(X_test_multilabel)

precision15 = precision_score(y_test_multilabel, prediction15, average='micro')

recall15 = recall_score(y_test_multilabel, prediction15, average='micro')

f1_score15 = 2*((precision15 * recall15)/(precision15 + recall15))

print("precision15: {:.4f}, recall15: {:.4f}, F1-measure: {:.4f}".format(precision15, recall15, f1_score15))

precision15: 0.4427, recall15: 0.6486, F1-measure: 0.5262


In [54]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction15[k])[0],"\n")

Movie:  What's Your Raashee?
Actual genre:  prank
Predicted tag:  ['flashback' 'romantic'] 

Movie:  Sick Girl
Actual genre:  revenge, murder
Predicted tag:  ['cult' 'flashback' 'murder' 'violence'] 

Movie:  Slightly Scarlet
Actual genre:  murder
Predicted tag:  ['flashback' 'murder' 'romantic' 'violence'] 

Movie:  Istanbul United
Actual genre:  violence
Predicted tag:  ['romantic'] 

Movie:  The Last Legion
Actual genre:  good versus evil, action
Predicted tag:  ['flashback' 'murder' 'violence'] 



<h2> 4.4 OneVsRestClassifier + LogisticRegression:</h2>

In [55]:
lr = LogisticRegression(class_weight='balanced')

clf = OneVsRestClassifier(lr)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=LogisticRegression(class_weight='balanced'))

In [56]:
prediction16 = clf.predict(X_test_multilabel)

precision16 = precision_score(y_test_multilabel, prediction16, average='micro')

recall16 = recall_score(y_test_multilabel, prediction16, average='micro')

f1_score16 = 2*((precision16 * recall16)/(precision16 + recall16))

print("precision16: {:.4f}, recall16: {:.4f}, F1-measure: {:.4f}".format(precision16, recall16, f1_score16))

precision16: 0.4469, recall16: 0.6676, F1-measure: 0.5354


In [57]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction16[k])[0],"\n")

Movie:  The Minus Man
Actual genre:  violence, murder, home movie
Predicted tag:  ['cult' 'murder' 'violence'] 

Movie:  Hotel Rwanda
Actual genre:  mystery, cruelty, murder, depressing, dramatic, violence, good versus evil, tragedy, romantic, historical
Predicted tag:  ['murder'] 

Movie:  Cliffhanger
Actual genre:  mystery, suspenseful, cult, violence, good versus evil, action, tragedy, revenge
Predicted tag:  ['cult' 'murder' 'violence'] 

Movie:  The Road Killers
Actual genre:  revenge, cult, murder, sadist
Predicted tag:  ['cult' 'murder' 'violence'] 

Movie:  Forrest Gump
Actual genre:  comedy, psychological, stupid, whimsical, dramatic, cult, cute, clever, flashback, historical fiction, psychedelic, humor, action, feel-good, melodrama, inspiring, romantic, historical, entertaining, sentimental
Predicted tag:  ['flashback' 'romantic'] 



<h1> Conclusion: </h1>

In [58]:
from prettytable import PrettyTable

tabel = PrettyTable()

tabel.field_names=['Model','Vectorizer','ngrams','Precision','recall','f1_score']



tabel.add_row(['MultinomialNB', 'TfidfVectorizer', '(1, 1)', round(precision1, 3),round(recall1, 3), round(f1_score1, 3)])

tabel.add_row(['SGDClassifier(log)', 'TfidfVectorizer', '(1, 1)', round(precision2, 3), round(recall2, 3), round(f1_score2, 3)])

tabel.add_row(['SGDClassifier(hinge)','TfidfVectorizer','(1, 1)' ,round(precision3, 3),  round(recall3, 3), round(f1_score3, 3)])

tabel.add_row(['LogisticRegression','TfidfVectorizer','(1, 1)', round(precision4, 3), round(recall4, 3), round(f1_score4, 3)])

tabel.add_row(['','','','','',''])
tabel.add_row(['','','','','',''])


tabel.add_row(['MultinomialNB', 'TfidfVectorizer', '(1, 2)', round(precision5, 3), round(recall5, 3),  round(f1_score5, 3)])

tabel.add_row(['SGDClassifier(log)', 'TfidfVectorizer', '(1, 2)', round(precision6, 3),  round(recall6, 3),  round(f1_score6, 3)])

tabel.add_row(['SGDClassifier(hinge)','TfidfVectorizer','(1, 2)', round(precision7, 3),  round(recall7, 3),  round(f1_score7, 3)])

tabel.add_row(['LogisticRegression','TfidfVectorizer','(1, 2)', round(precision8, 3),  round(recall8, 3),  round(f1_score8, 3)])

tabel.add_row(['','','','','',''])
tabel.add_row(['','','','','',''])


tabel.add_row(['MultinomialNB', 'TfidfVectorizer', '(1, 3)', round(precision9, 3),  round(recall9, 3), round(f1_score9, 3)])

tabel.add_row(['SGDClassifier(log)', 'TfidfVectorizer', '(1, 3)', round(precision10, 3),  round(recall10, 3), round(f1_score10, 3)])

tabel.add_row(['SGDClassifier(hinge)','TfidfVectorizer','(1, 3)', round(precision11, 3),  round(recall11, 3), round(f1_score11, 3)])

tabel.add_row(['LogisticRegression','TfidfVectorizer','(1, 3)', round(precision12, 3),  round(recall12, 3), round(f1_score12, 3)])

tabel.add_row(['','','','','',''])
tabel.add_row(['','','','','',''])

tabel.add_row(['MultinomialNB', 'TfidfVectorizer', '(1, 4)', round(precision13, 3), round(recall13, 3), round(f1_score13, 3)])

tabel.add_row(['SGDClassifier(log)', 'TfidfVectorizer', '(1, 4)', round(precision14, 3), round(recall14, 3), round(f1_score14, 3)])

tabel.add_row(['SGDClassifier(hinge)','TfidfVectorizer','(1, 4)', round(precision15, 3), round(recall15, 3), round(f1_score15, 3)])

tabel.add_row(['LogisticRegression','TfidfVectorizer','(1, 4)', round(precision16, 3),  round(recall16, 3),  round(f1_score16, 3)])



print(tabel)

+----------------------+-----------------+--------+-----------+--------+----------+
|        Model         |    Vectorizer   | ngrams | Precision | recall | f1_score |
+----------------------+-----------------+--------+-----------+--------+----------+
|    MultinomialNB     | TfidfVectorizer | (1, 1) |   0.411   | 0.676  |  0.511   |
|  SGDClassifier(log)  | TfidfVectorizer | (1, 1) |   0.448   | 0.676  |  0.539   |
| SGDClassifier(hinge) | TfidfVectorizer | (1, 1) |   0.435   | 0.674  |  0.529   |
|  LogisticRegression  | TfidfVectorizer | (1, 1) |   0.447   | 0.668  |  0.535   |
|                      |                 |        |           |        |          |
|                      |                 |        |           |        |          |
|    MultinomialNB     | TfidfVectorizer | (1, 2) |   0.412   | 0.676  |  0.512   |
|  SGDClassifier(log)  | TfidfVectorizer | (1, 2) |   0.455   |  0.66  |  0.539   |
| SGDClassifier(hinge) | TfidfVectorizer | (1, 2) |   0.441   | 0.672  |  0.

<h1>5. Word2Vec</h1>

In [59]:
X_train_new = []

for i in tqdm(range(len(list(X_train)))):
    X_train_new.append(X_train[i].split(" "))

100%|██████████████████████████████████████████████████████████████████████████| 10989/10989 [00:01<00:00, 7231.48it/s]


In [62]:
with open('pickled glove 6b 300d.pkl', 'rb') as f:
    new_model = pickle.load(f)
    words =  set(new_model.keys())

In [63]:
X_train_multilabel = []; # the avg-w2v for each sentence/review is stored in this list
for sentence in tqdm(X_train.values): # for each review/sentence
    vector = np.zeros(300) # as word vectors are of zero length
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sentence.split():
        if word in words:
            vector += new_model[word]
            cnt_words += 1
    if cnt_words != 0:
        vector /= cnt_words
    X_train_multilabel.append(vector)

100%|███████████████████████████████████████████████████████████████████████████| 10989/10989 [00:21<00:00, 512.94it/s]


In [64]:
X_test_multilabel = []; # the avg-w2v for each sentence/review is stored in this list
for sentence in tqdm(X_test.values): # for each review/sentence
    vector = np.zeros(300) # as word vectors are of zero length
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sentence.split():
        if word in words:
            vector += new_model[word]
            cnt_words += 1
    if cnt_words != 0:
        vector /= cnt_words
    X_test_multilabel.append(vector)

100%|█████████████████████████████████████████████████████████████████████████████| 2768/2768 [00:07<00:00, 348.74it/s]


<H2> 5.1 OneVsRestClassifier + SGDClassifier with LOG Loss :</H2>

In [65]:
sgl = SGDClassifier(loss='log', class_weight='balanced')

clf = OneVsRestClassifier(sgl)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(class_weight='balanced',
                                            loss='log'))

In [66]:
prediction17 = clf.predict(X_test_multilabel)

precision17 = precision_score(y_test_multilabel, prediction17, average='micro')

recall17 = recall_score(y_test_multilabel, prediction17, average='micro')

f1_score17 = 2*((precision17 * recall17)/(precision17 + recall17))

print("precision17: {:.4f}, recall17: {:.4f}, F1-measure: {:.4f}".format(precision17, recall17, f1_score17))

precision17: 0.4103, recall17: 0.6801, F1-measure: 0.5119


In [67]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction17[k])[0],"\n")

Movie:  Last of the Dogmen
Actual genre:  comedy, flashback
Predicted tag:  ['cult' 'murder' 'violence'] 

Movie:  Some Like It Hot
Actual genre:  entertaining, cult, comedy, murder, romantic
Predicted tag:  [] 

Movie:  I'm Not There.
Actual genre:  cruelty, anti war, cult, psychedelic, satire, romantic, revenge
Predicted tag:  ['cult' 'flashback' 'romantic'] 

Movie:  Senjo no Varukyuria: Garian keronikeruzu
Actual genre:  tragedy
Predicted tag:  ['violence'] 

Movie:  Thunder Afloat
Actual genre:  revenge
Predicted tag:  ['cult'] 



<h2> 5.2 OneVsRestClassifier + SGDClassifier with HINGE Loss : </h2>

In [68]:
sgh = SGDClassifier(loss='hinge', class_weight='balanced')

clf = OneVsRestClassifier(sgh)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(class_weight='balanced'))

In [69]:
prediction18 = clf.predict(X_test_multilabel)

precision18 = precision_score(y_test_multilabel, prediction18, average='micro')

recall18 = recall_score(y_test_multilabel, prediction18, average='micro')

f1_score18 = 2*((precision18 * recall18)/(precision18 + recall18))

print("precision18: {:.4f}, recall18: {:.4f}, F1-measure: {:.4f}".format(precision18, recall18, f1_score18))

precision18: 0.3975, recall18: 0.6944, F1-measure: 0.5056


In [70]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction18[k])[0],"\n")

Movie:  Santa sangre
Actual genre:  avant garde, mystery, murder, suspenseful, thought-provoking, cult, violence, atmospheric, absurd, psychedelic, revenge, entertaining
Predicted tag:  ['cult' 'flashback' 'murder' 'violence'] 

Movie:  System Shock
Actual genre:  sci-fi
Predicted tag:  ['cult' 'flashback' 'violence'] 

Movie:  Child's Play 3
Actual genre:  violence, comedy, gothic, dark, storytelling
Predicted tag:  ['cult' 'violence'] 

Movie:  Wrong Side of Town
Actual genre:  revenge
Predicted tag:  ['murder' 'violence'] 

Movie:  Cameron's Closet
Actual genre:  paranormal
Predicted tag:  ['cult' 'flashback' 'murder' 'violence'] 



<h2> 5.3 OneVsRestClassifier + LogisticRegression:</h2>

In [71]:
lr = LogisticRegression(class_weight='balanced')

clf = OneVsRestClassifier(lr)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=LogisticRegression(class_weight='balanced'))

In [72]:
prediction19 = clf.predict(X_test_multilabel)

precision19 = precision_score(y_test_multilabel, prediction19, average='micro')

recall19 = recall_score(y_test_multilabel, prediction19, average='micro')

f1_score19 = 2*((precision19 * recall19)/(precision19 + recall19))

print("precision19: {:.4f}, recall19: {:.4f}, F1-measure: {:.4f}".format(precision19, recall19, f1_score19))

precision19: 0.4183, recall19: 0.6740, F1-measure: 0.5162


In [73]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction19[k])[0],"\n")

Movie:  Wilby Wonderful
Actual genre:  comedy
Predicted tag:  ['flashback'] 

Movie:  How to Steal a Million
Actual genre:  comedy
Predicted tag:  ['cult'] 

Movie:  Shuttle
Actual genre:  murder
Predicted tag:  ['cult' 'flashback' 'murder' 'violence'] 

Movie:  The Son of No One
Actual genre:  mystery, boring, murder, flashback
Predicted tag:  ['flashback' 'murder' 'violence'] 

Movie:  The Virgin Queen
Actual genre:  romantic, intrigue
Predicted tag:  ['romantic'] 



<h1>Conclusion</h1>

In [74]:
from prettytable import PrettyTable

tabel = PrettyTable()

tabel.field_names=['Model', 'Vectorizer', 'Precision','recall','f1_score']


tabel.add_row(['SGDClassifier(log)', 'AVG W2V',  round(precision17, 3), round(recall17, 3), round(f1_score17, 3)])

tabel.add_row(['SGDClassifier(hinge)','AVG W2V', round(precision18, 3),  round(recall18, 3), round(f1_score18, 3)])

tabel.add_row(['LogisticRegression','AVG W2V', round(precision19, 3), round(recall19, 3), round(f1_score19, 3)])


print(tabel)

+----------------------+------------+-----------+--------+----------+
|        Model         | Vectorizer | Precision | recall | f1_score |
+----------------------+------------+-----------+--------+----------+
|  SGDClassifier(log)  |  AVG W2V   |    0.41   |  0.68  |  0.512   |
| SGDClassifier(hinge) |  AVG W2V   |   0.398   | 0.694  |  0.506   |
|  LogisticRegression  |  AVG W2V   |   0.418   | 0.674  |  0.516   |
+----------------------+------------+-----------+--------+----------+
